In [3]:
# import modules 
import requests 
import pandas as pd 

# create a request header 
headers = {'User-Agent': "hbirring@seattleu.edu"}

# get all companies data
companyTickers = requests.get("https://www.sec.gov/files/company_tickers.json", headers=headers)

# print(companyTickers.json()['0']['cik_str'])

companyCIK = pd.DataFrame.from_dict(companyTickers.json(), orient= 'index')

print(companyCIK)

companyCIK['cik_str']= companyCIK['cik_str'].astype(str).str.zfill(10)

print(companyCIK)

companyCIK = companyCIK.set_index('ticker')

alphabet_cik = companyCIK.at['GOOGL', 'cik_str']

print(alphabet_cik)

# get all companies data
companyFacts = requests.get(f"https://data.sec.gov/api/xbrl/companyfacts/CIK{alphabet_cik}.json", headers=headers)

print(companyFacts.json().keys())

       cik_str ticker                              title
0      1045810   NVDA                        NVIDIA CORP
1       789019   MSFT                     MICROSOFT CORP
2       320193   AAPL                         Apple Inc.
3      1018724   AMZN                     AMAZON COM INC
4      1652044  GOOGL                      Alphabet Inc.
...        ...    ...                                ...
10055  1949257  RDPTF        Radiopharm Theranostics Ltd
10056  1733968  WLGMF      West Red Lake Gold Mines Ltd.
10057  1991946  CGBSW             Crown LNG Holdings Ltd
10058  2038919  DWWYF        Dowway Holdings Limited/ADR
10059  2042556  SPGDF  Springer Nature AG & Co. KGaA/ADR

[10060 rows x 3 columns]
          cik_str ticker                              title
0      0001045810   NVDA                        NVIDIA CORP
1      0000789019   MSFT                     MICROSOFT CORP
2      0000320193   AAPL                         Apple Inc.
3      0001018724   AMZN                     AMAZO

In [5]:
# gross profit margin & net profit margin 
# net profit margin = net income / revenue 
# gross profit margin = gross profit / revenue 

import requests 
import pandas as pd 

# config of the connection via CIK number 
CIK_googl = '0001652044' # Googl Alphabet Inc. 
headers = {'User-Agent': "hbirring@seattleu.edu"}

url = f'http://data.sec.gov/api/xbrl/companyfacts/CIK{CIK_googl.zfill(10)}.json'
companyFacts = requests.get(url, headers=headers).json()

#tag the data
def get_tag_df(tag):
    try:
        return pd.DataFrame(companyFacts['facts']['us-gaap'][tag]['units']['USD'])
    except KeyError:
        return pd.DataFrame(columns=['accn', 'end', 'val', 'form', 'fy'])
    
#load the dataframes
revenues_df = get_tag_df('Revenues')
net_income_df = get_tag_df('NetIncomeLoss')
cost_of_revenue_df = get_tag_df('CostOfRevenue')

# filter the dataframes for the 10-k filings
revenues_df = revenues_df[revenues_df['form'] == '10-K']
net_income_df = net_income_df[net_income_df['form'] == '10-K']
cost_of_revenue_df = cost_of_revenue_df[cost_of_revenue_df['form'] == '10-K']

# get the most recent filings
revenues_df = revenues_df.sort_values('end').drop_duplicates('accn', keep= 'last')
net_income_df = net_income_df.sort_values('end').drop_duplicates('accn', keep= 'last')
cost_of_revenue_df = cost_of_revenue_df.sort_values('end').drop_duplicates('accn', keep= 'last')

#merge all dataframes on 'accn' and 'end' columns
df = revenues_df[['accn', 'end', 'val']].rename(columns={'val': 'revenue'})
df = df.merge(net_income_df[['accn', 'end', 'val']].rename(columns={'val': 'net_income'}), on=['accn', 'end'])
df = df.merge(cost_of_revenue_df[['accn', 'end', 'val']].rename(columns={'val': 'cost_of_revenue'}), on=['accn', 'end'])

# calculate the net profit & gross profit margins 
df['gross_profit'] = df['revenue'] - df['cost_of_revenue']
df['gross_profit_margin'] = df['gross_profit'] / df['revenue']
df['net_profit_margin'] = df['net_income'] / df['revenue']

#print out the results 
print(df[['end', 'revenue', 'net_income', 'cost_of_revenue', 'gross_profit', 'gross_profit_margin', 'net_profit_margin']])



          end       revenue   net_income  cost_of_revenue  gross_profit  \
0  2015-12-31   74989000000  16348000000      28164000000   46825000000   
1  2016-12-31   90272000000  19478000000      35138000000   55134000000   
2  2017-12-31  110855000000  12662000000      45583000000   65272000000   
3  2020-12-31  182527000000  40269000000      84732000000   97795000000   
4  2021-12-31  257637000000  76033000000     110939000000  146698000000   

   gross_profit_margin  net_profit_margin  
0             0.624425           0.218005  
1             0.610754           0.215770  
2             0.588805           0.114221  
3             0.535784           0.220619  
4             0.569398           0.295117  
